
# 🏆 Scraper les titres Transfermarkt 2024–2025
Ce notebook automatise la récupération des titres récents (2023–2025) pour une liste de joueurs via leurs pages `transfermarkt.com`.


In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}


In [ ]:

def get_transfermarkt_url(player_name):
    query = player_name.replace(" ", "+")
    search_url = f"https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query={query}"
    r = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    result = soup.select_one("table.items tbody tr td.hauptlink a")
    if result:
        return "https://www.transfermarkt.com" + result["href"].replace("/profil", "/erfolge")
    return None


In [ ]:

def get_titles(player_url, player_name):
    r = requests.get(player_url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.find("table", class_="erfolg_table")
    titles = []

    if not table:
        return titles

    rows = table.find_all("tr")[1:]
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 3:
            season = cols[0].text.strip()
            title = cols[1].text.strip()
            club = cols[2].text.strip()
            if "2023" in season or "2024" in season or "2025" in season:
                titles.append({
                    "Player": player_name,
                    "Season": season,
                    "Title": title,
                    "Club": club
                })
    return titles


In [ ]:

players = ['Federico Valverde', 'Kylian Mbappé', 'Jude Bellingham', 'Vinicius Júnior', 'Luka Modrić',
           'Eduardo Camavinga', 'Dani Carvajal', 'Rodrygo', 'Antonio Rüdiger', 'Andriy Lunin', 'Toni Kroos',
           'Pedri', 'Lamine Yamal', 'Alejandro Balde', 'Robert Lewandowski', 'Nacho', 'Joselu',
           'Aurélien Tchouaméni', 'Fran García', 'Gavi', 'Raphinha', 'Erling Haaland', 'Phil Foden',
           'Rodri', 'Florian Wirtz', 'Harry Kane', 'João Cancelo', 'Achraf Hakimi', 'Lionel Messi']

all_titles = []

for player in players:
    print(f"🔍 Recherche de {player}...")
    url = get_transfermarkt_url(player)
    if url:
        print(f"➡️  {url}")
        titles = get_titles(url, player)
        all_titles.extend(titles)
    else:
        print("❌ Aucune URL trouvée.")
    time.sleep(2)

df_titles = pd.DataFrame(all_titles)
df_titles.to_csv("scraped_titles_2025.csv", index=False)
df_titles.head()
